In [23]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
#from langchain_openai import ChatOpenAI
import operator
from typing import Annotated, TypedDict, Union
from langchain_core.agents import AgentAction, AgentFinish
from langgraph.prebuilt.tool_executor import ToolExecutor
from langchain import hub
from datetime import datetime
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
from langchain_openai.chat_models import ChatOpenAI
from langchain.llms import AzureOpenAI
from langchain.agents import create_react_agent
from typing import List, Sequence
import os
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import END, MessageGraph
from dotenv import load_dotenv
load_dotenv()

False

In [10]:
import operator
from typing import Annotated, TypedDict, Union

from langchain_core.agents import AgentAction, AgentFinish

from langchain_core.agents import AgentFinish
from langgraph.graph import END, StateGraph
from dotenv import load_dotenv
from langgraph.prebuilt.tool_executor import ToolExecutor
from dotenv import load_dotenv
from langchain import hub
from langchain.agents import create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
from langchain_openai.chat_models import ChatOpenAI

In [8]:
class AgentState(TypedDict): # stores the trajectory
    input: str
    agent_outcome: Union[AgentAction, AgentFinish, None] #the outcome could be either Agent Action type or AgentFinish type
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add] # check the notes on State Graph below for explanation

# Notes on "typing" library

# ReAct Prompt

In [21]:

react_prompt: PromptTemplate = hub.pull("hwchase17/react")
print(react_prompt)

C:\Users\saura\Anaconda3\envs\office\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'


# Tools 

In [25]:
@tool
def triple(num: float) -> float:
    """
    :param num: a number to triple
    :return: the number tripled ->  multiplied by 3
    """
    return 3 * float(num)
@tool
def return_current_time() -> datetime :
    """"
    Use this function to return the current time ( in the hour, minute, second format)
    input param : None
    return : current time in the format '%H:%M:%S'
    """
    # storing the current time in the variable
    c = datetime.now()
    
    # Displays Time
    current_time = c.strftime('%H:%M:%S')
    return current_time

In [ ]:
tools = [TavilySearchResults(max_results=1), triple]

llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

react_agent_runnable = create_react_agent(llm, tools, react_prompt)

C:\Users\saura\Anaconda3\envs\office\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [26]:

def run_agent_reasoning_engine(state: AgentState):
    agent_outcome = react_agent_runnable.invoke(state)
    return {"agent_outcome": agent_outcome}


tool_executor = ToolExecutor(tools)


def execute_tools(state: AgentState):
    agent_action = state["agent_outcome"]
    output = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(output))]}


C:\Users\saura\AppData\Local\Temp\ipykernel_29692\1535337541.py:6: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_executor = ToolExecutor(tools)


In [27]:

AGENT_REASON = "agent_reason"
ACT = "act"


def should_continue(state: AgentState) -> str:
    if isinstance(state["agent_outcome"], AgentFinish):
        return END
    else:
        return ACT


flow = StateGraph(AgentState)
# graph.add_node(name, value) syntax. The value parameter should be either a function or LCEL runnable that will be called
# This function/LCEL should accept a Dict in the same form as the State object as input, and output a Dict with keys of the State object to update.
flow.add_node(AGENT_REASON, run_agent_reasoning_engine)
flow.set_entry_point(AGENT_REASON)
flow.add_node(ACT, execute_tools)


flow.add_conditional_edges(
    AGENT_REASON,
    should_continue,
)

flow.add_edge(ACT, AGENT_REASON)

app = flow.compile()
app.get_graph().draw_mermaid_png(output_file_path="graph.png")

if __name__ == "__main__":
    print("Hello ReAct with LangGraph")
    res = app.invoke(
        input={
            "input": "what is the weather in sf? List it and then Triple it ",
        }
    )
    print(res["agent_outcome"].return_values["output"])


Hello ReAct with LangGraph
The current weather in San Francisco is 10.3 degrees Celsius, and when tripled, it is 30.9 degrees Celsius.


In [28]:
res["agent_outcome"]

AgentFinish(return_values={'output': 'The current weather in San Francisco is 10.3 degrees Celsius, and when tripled, it is 30.9 degrees Celsius.'}, log='I now know the final answer\nFinal Answer: The current weather in San Francisco is 10.3 degrees Celsius, and when tripled, it is 30.9 degrees Celsius.')

In [30]:
res['intermediate_steps']

[(AgentAction(tool='tavily_search_results_json', tool_input='current weather in San Francisco', log='I should use the search engine to find the current weather in San Francisco, and then use the triple function to triple the temperature.\nAction: tavily_search_results_json\nAction Input: "current weather in San Francisco"'),
  '[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.775, \'lon\': -122.4183, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1733144329, \'localtime\': \'2024-12-02 04:58\'}, \'current\': {\'last_updated_epoch\': 1733143500, \'last_updated\': \'2024-12-02 04:45\', \'temp_c\': 10.3, \'temp_f\': 50.5, \'is_day\': 0, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/116.png\', \'code\': 1003}, \'wind_mph\': 5.4, \'wind_kph\': 8.6, \'wind_degree\': 42, \'wind_dir\': \'NE\', \'pr

### Since we only have the triple function and not LLM math, we are incorrectly calling it ---returning 27 in stead of 81

In [32]:
if __name__ == "__main__":
    print("Hello ReAct with LangGraph")
    res = app.invoke(
        input={
            "input": "what is the name of Aizen's Shikai ? Tell me the name and also the square of 9 ",
        }
    )
    print(res["agent_outcome"].return_values["output"])

Hello ReAct with LangGraph
The name of Aizen's Shikai is Kyouka Suigetsu and the square of 9 is 27.


In [33]:
res['intermediate_steps']

[(AgentAction(tool='tavily_search_results_json', tool_input="Aizen's Shikai name", log='I need to search for the name of Aizen\'s Shikai and calculate the square of 9\nAction: tavily_search_results_json\nAction Input: "Aizen\'s Shikai name"'),
  '[{\'url\': \'https://bleach.fandom.com/wiki/Sōsuke_Aizen\', \'content\': "When Aizen noted that the Hōgyoku in its half-awakened state can be fully activated when temporarily fusing with someone with at least twice as much spiritual power as an average captain-level Shinigami, he calmly proved able to do so himself.[226] After the deaths of Baraggan Louisenbairn and Coyote Starrk and Aizen subsequently cutting down Tier Harribel, he claimed to be more powerful than all of the Espada combined.[227] During his battle with the Gotei 13 and the Visored, Aizen was capable of defeating several of his opponents at the same time relatively unscathed, with Gin Ichimaru commenting that no amount of preparation could hope to combat Aizen.[222] After spen

In [35]:
res = app.invoke(
        input={
            "input": "what is the current time? Tell me the hour and multiply by 3 ",
        }
    )
print(res["agent_outcome"].return_values["output"])

ValidationError: 1 validation error for triple
num
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='current hour', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/float_parsing

In [ ]:
ValidationError: 1 validation error for triple
num
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='current hour', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/float_parsing

## Use case where ReAct failed

In [18]:
if __name__ == "__main__":
    print("Hello ReAct with LangGraph")
    res = app.invoke(
        input={
            "input": "what is the name of Aizen's Shikai ? Tell me the name and also the square of 1+2+3*2 ",
        }
    )
    print(res["agent_outcome"].return_values["output"])


Hello ReAct with LangGraph


ValidationError: 1 validation error for triple
num
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='1+2+3*2', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/float_parsing

ValidationError: 1 validation error for triple
num
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='1+2+3*2', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/float_parsing

# Implementation 2